# Expansão de Consultas (Lab 05)  - Recuperação da Informação
**Aluno: Pedro Guedes Braga**


In [129]:
#coding: utf-8

import pandas as pd
import nltk
import operator
import math
import matplotlib.pyplot as pyplot
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import *
nltk.download('stopwords')
nltk.download('punkt') 
import csv
import time
import numpy
from operator import itemgetter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**1 - Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que? (20 pts.)**




* **Observando as tabelas, se pode observar que a métrica "EMIM" - Expected Mutual Information se saiu melhor, porém há de ressaltar que as metricas X^2 e "Coeficiente de Dice" tiveram resultados muito similares. A única métrica que ficou distante das demais foi a "MIM"**.

In [0]:
url_csv = "https://raw.githubusercontent.com/PedroGuedesBraga/ri_lab_01/master/output/results.csv"
df = pd.read_csv(url_csv)
df.dropna(inplace=True)  ##Retira campos NA (ex.: NaN, etc)
lista_de_noticias = df.text.unique() #Retorna uma lista onde cada elemento é o texto de uma noticia.
 # http://www.nltk.org/howto/portuguese_en.html - Tutorial StopWords
 # https://pythonspot.com/tokenizing-words-and-sentences-with-nltk/
palavras_a_ignorar = nltk.corpus.stopwords.words('portuguese')
palavras_a_ignorar.append('o')
palavras_a_ignorar.append('a')
palavras_a_ignorar.append('é')
palavras_a_ignorar.append('r')
  
special_chars = [',', '.', '-', '“', "”", ")", "(", ":", "%", "?", "$", "–", ""]



##Auxiliares
##Recebe um token(ja em lowercase) e aplica mais estratégias de tokenizacao, descritas pelas duas funcoes auxiliares acima.
def apply_tokenize_strategy(token):
  return remove_periods((remove_apostrophes(token)))

##Metodo auxiliar que remove apostrofos dos tokens, retornando um token em seu formato sem apostrofo
def remove_apostrophes(token):
  return token.replace("’", "").replace("'","")
  
##Metodo auxiliar que remove pontos em uma abreviacao
def remove_periods(token):
  return token.replace(".", "")

#Funcao auxiliar, que simplesmente resgata todos os documentos no indice
def getAllDocuments(index):
  list_of_documents = []
  for word in index:
    for tupla in index[word]:
      if(tupla[0] not in list_of_documents):
        list_of_documents.append(tupla[0])
        
  return list_of_documents





##Questao 1
##Funcao para construir o índice:
def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n = n + 1
    tokens = word_tokenize(document)
    for token in tokens:
      if((apply_tokenize_strategy(token.lower()) not in palavras_a_ignorar) and (apply_tokenize_strategy(token.lower()) not in special_chars) and ("," not in token.lower())):  ## Todos os tokens passam a serem tratados em LOWERCASE
        if(apply_tokenize_strategy(token.lower()) in index):
          index[apply_tokenize_strategy(token.lower())].append((n, tokens.count(token)))
        else:
          index[apply_tokenize_strategy(token.lower())] = [(n, tokens.count(token))]
  refactored_index = aux_refactor_index(index)
  return refactored_index


#funcao auxiliar responsavel por refatorar as listas invertidas do indice para retorna-la no formato pedido. - Remove duplicatas de documentos
def aux_refactor_index(index):
  for word in index:
    for tupla in index[word]:
      index[word] = set(index[word])
  return index
    

#Constrói o index
indice = build_index(lista_de_noticias)


############################################################
####################### AUXILIARES #########################
############################################################

#Conta o numero de documentos que contém ambos os termos 'a' e 'b ao mesmo tempo'
def number_docs_contain_both_terms(a, b):
  documents_containing_a = [i[0] for i in indice[a]]
  documents_containing_b = [i[0] for i in indice[b]]
  contador = 0
  for doc_contain_a in documents_containing_a:
    for doc_contain_b in documents_containing_b:
      if(doc_contain_a == doc_contain_b):
        contador = contador + 1
  return contador



def number_docs_containing_a_term(term):
  return len(indice[term])




#############################################################


#############################################################
############# MEDIDAS DE ASSOCIACAO DE TERMOS ###############
#############################################################

n = len(getAllDocuments(indice))

# Calcula o coeficiente de Dice dados os termos 'a' e 'b' como parametros
def dice_coefficient(a, b):
  n_ab = number_docs_contain_both_terms(a, b)
  n_a = number_docs_containing_a_term(a)
  n_b = number_docs_containing_a_term(b)
  dice_coefficient = (2 * n_ab) / (n_a + n_b)
  return dice_coefficient

def mutual_information(a, b):
  n_ab = number_docs_contain_both_terms(a, b)
  n_a = number_docs_containing_a_term(a)
  n_b = number_docs_containing_a_term(b)
  mutual_information = n_ab / (n_a * n_b)
  return mutual_information

def expected_mutual_information(a, b):
  n_ab = number_docs_contain_both_terms(a, b)
  n_a = number_docs_containing_a_term(a)
  n_b = number_docs_containing_a_term(b)
  if ((n_a * n_b) != 0):
    t1 = n * (n_ab / (n_a * n_b))
  else: 
    t1 = 0
  if(t1 == 0):
    return 0
  emim = n_ab * numpy.log(t1)
  return emim

def chi_square(a, b):
  n_ab = number_docs_contain_both_terms(a, b)
  n_a = number_docs_containing_a_term(a)
  n_b = number_docs_containing_a_term(b)
  return (n_ab - (1/n) * n_a * n_b)**2 / (n_a * n_b)

#################CRIANDO AS LISTAS##############################33
#Cria a lista de palavras mais associadas com termo, usando o algoritmo do coeficiente de dice
def build_list_using_dice_coefficient(term):
  result = [] #Lista de tuplas do tipo: [(palavra, medida de associacao)]
  for entry in indice:
    if(entry != term):
      result.append((entry, dice_coefficient(term, entry)))
  return result

#Cria a lista de palavras mais associadas com termo, usando o algoritmo do chi_squared
def build_list_using_chi_squared(term):
  result = [] #Lista de tuplas do tipo: [(palavra, medida de associacao)]
  for entry in indice:
    if(entry != term):
      result.append((entry, chi_square(term, entry)))
  return result

#Cria a lista de palavras mais associadas com termo, usando o algoritmo de MIM
def build_list_using_mim(term):
  result = [] #Lista de tuplas do tipo: [(palavra, medida de associacao)]
  for entry in indice:
    if(entry != term):
      result.append((entry, mutual_information(term, entry)))
  return result


#Cria a lista de palavras mais associadas com termo, usando o algoritmo EMIM
def build_list_using_emim(term):
  result = [] #Lista de tuplas do tipo: [(palavra, medida de associacao)]
  for entry in indice:
    if(entry != term):
      result.append((entry, expected_mutual_information(term, entry)))
  return result



**Calculando as TOP 10 palavras mais associadas a cada consulta de um termo**

 Termos:  ["bolsonaro", "presidente", "lula", "golpe", "economia"]

Termo "**bolsonaro**":

In [131]:
consultas = ["bolsonaro", "presidente", "lula", "golpe", "economia"]

top_words_using_dice_first_term = sorted(build_list_using_dice_coefficient(consultas[0]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_chi_squared_first_term = sorted(build_list_using_chi_squared(consultas[0]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_mim_first_term = sorted(build_list_using_mim(consultas[0]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_emim_first_term = sorted(build_list_using_emim(consultas[0]), key = itemgetter(1), reverse = True)[0:10]

pd.DataFrame({
    'MIM': [x1 for (x1,x2) in top_words_using_mim_first_term],
    'EMIM': [x1 for (x1,x2) in top_words_using_emim_first_term],
    'X²': [x1 for (x1, x2) in top_words_using_chi_squared_first_term],
    'Dice': [x1 for (x1, x2) in top_words_using_dice_first_term]
})





,MIM,EMIM,X²,Dice
0,coordenação,jair,jair,jair
1,adjacentes,governo,governo,governo
2,normalmente,brasil,armadas,brasil
3,rápidas,presidente,psl,presidente
4,subitamente,país,presidente,país
5,propendem,parte,recorda,ser
6,positivas,psl,perseguidos,política
7,quo,reforma,antifascistas,sobre
8,instalando,vai,forças,hoje
9,paulatinamente,política,brasil,vai


Termo **presidente**:

In [132]:
top_words_using_dice_first_term = sorted(build_list_using_dice_coefficient(consultas[1]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_chi_squared_first_term = sorted(build_list_using_chi_squared(consultas[1]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_mim_first_term = sorted(build_list_using_mim(consultas[1]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_emim_first_term = sorted(build_list_using_emim(consultas[1]), key = itemgetter(1), reverse = True)[0:10]

pd.DataFrame({
    'MIM': [x1 for (x1,x2) in top_words_using_mim_first_term],
    'EMIM': [x1 for (x1,x2) in top_words_using_emim_first_term],
    'X²': [x1 for (x1, x2) in top_words_using_chi_squared_first_term],
    'Dice': [x1 for (x1, x2) in top_words_using_dice_first_term]
})

,MIM,EMIM,X²,Dice
0,1956,bolsonaro,vice-presidente,bolsonaro
1,fontella,política,assumiu,política
2,acompanhando,governo,república,governo
3,thereza,jair,políticos,jair
4,fundadores,assim,boa,brasil
5,leonel,contra,política,contra
6,rollemberg,república,democracia,assim
7,recusa,políticos,agrária,país
8,herdeira,parte,bolsonaro,sobre
9,mr-8,reforma,comissão,ainda


Termo **lula**:

In [133]:
top_words_using_dice_first_term = sorted(build_list_using_dice_coefficient(consultas[2]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_chi_squared_first_term = sorted(build_list_using_chi_squared(consultas[2]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_mim_first_term = sorted(build_list_using_mim(consultas[2]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_emim_first_term = sorted(build_list_using_emim(consultas[2]), key = itemgetter(1), reverse = True)[0:10]

pd.DataFrame({
    'MIM': [x1 for (x1,x2) in top_words_using_mim_first_term],
    'EMIM': [x1 for (x1,x2) in top_words_using_emim_first_term],
    'X²': [x1 for (x1, x2) in top_words_using_chi_squared_first_term],
    'Dice': [x1 for (x1, x2) in top_words_using_dice_first_term]
})

,MIM,EMIM,X²,Dice
0,ithié,ex-presidente,ex-presidente,ex-presidente
1,revogou,jato,jato,jato
2,investigados,luiz,luiz,políticos
3,vala,políticos,voto,pt
4,justiciamento,pt,olhos,luiz
5,voga,eleitoral,julgado,partido
6,impetrado,partido,inconstitucional,prisão
7,colegiada,voto,antemão,eleitoral
8,bastou,prisão,incentivos,contrário
9,apurada,judicial,independência,brasileiro


Termo "**golpe**":

In [134]:
top_words_using_dice_first_term = sorted(build_list_using_dice_coefficient(consultas[3]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_chi_squared_first_term = sorted(build_list_using_chi_squared(consultas[3]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_mim_first_term = sorted(build_list_using_mim(consultas[3]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_emim_first_term = sorted(build_list_using_emim(consultas[3]), key = itemgetter(1), reverse = True)[0:10]

pd.DataFrame({
    'MIM': [x1 for (x1,x2) in top_words_using_mim_first_term],
    'EMIM': [x1 for (x1,x2) in top_words_using_emim_first_term],
    'X²': [x1 for (x1, x2) in top_words_using_chi_squared_first_term],
    'Dice': [x1 for (x1, x2) in top_words_using_dice_first_term]
})

,MIM,EMIM,X²,Dice
0,lutamos,1964,1964,1964
1,ferida,ditadura,64,ditadura
2,evocam,militar,desaparecidos,militar
3,cancelamento,64,55,democracia
4,cassação,democracia,comemorar,64
5,desencadearam,desaparecidos,ditadura,desaparecidos
6,greves,história,militar,história
7,afastamos,contra,democracia,brasileiros
8,pacificado,55,perseguições,tortura
9,sombrios,tortura,pacificação,contra


Termo **economia**:

In [135]:
top_words_using_dice_first_term = sorted(build_list_using_dice_coefficient(consultas[4]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_chi_squared_first_term = sorted(build_list_using_chi_squared(consultas[4]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_mim_first_term = sorted(build_list_using_mim(consultas[4]), key = itemgetter(1), reverse = True)[0:10]
top_words_using_emim_first_term = sorted(build_list_using_emim(consultas[4]), key = itemgetter(1), reverse = True)[0:10]

pd.DataFrame({
    'MIM': [x1 for (x1,x2) in top_words_using_mim_first_term],
    'EMIM': [x1 for (x1,x2) in top_words_using_emim_first_term],
    'X²': [x1 for (x1, x2) in top_words_using_chi_squared_first_term],
    'Dice': [x1 for (x1, x2) in top_words_using_dice_first_term]
})

,MIM,EMIM,X²,Dice
0,assassinatos,reforma,economista,previdência
1,manual,previdência,salarial,reforma
2,extrair,trabalho,lins,economista
3,confissões,economista,impactos,sistema
4,afogamentos,sistema,existiam,mínimo
5,sufocamentos,mínimo,aposta,população
6,prosperavam,população,péricles,salário
7,restrições,salário,sakamoto,trabalho
8,cidadã,modelo,fagnani,modelo
9,empobreceram,salarial,salário,crescimento


**2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta. (25 pts.)**

*  **Na medida em que aumentamos a quantidade de *top-documentos* para "pegar" os termos mais associados, a qualidade dos resultados aumenta, pois a como o vocabulário aumenta na medida em que aumentamos o número de documentos, a precisão também aumenta, pois o número de palavras aumenta. Em contrapartida, a performance diminui na medida em que aumentamos o numero de *top-documentos*.**

In [0]:
##Questao 2
##Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18). (2 pts)
##print(indice)

#No caso, so sao retornados os documentos que tem score > 0

def documentAtTimeRetrieval(query, index, k):
  query = query.lower() #Como na indexacao (no processo de tokenizacao) foi feito downcase, na consulta deve ser feito tbm
  inverted_lists = []
  query_terms = query.split(" ")
  documents = getAllDocuments(index)
  document_scores = [] #Uma lista de tuplas no formato (score_do_documento, documento)
  
  #Adiciona em 'inverted_lists' as listas invertidas que correspondentes a cada termo.
  for term in query_terms: 
    if(term in index):
      inverted_lists.append(index[term])
  
  
  for document in documents:
    score_doc = 0
    for inverted_list in inverted_lists:
      for tupla in inverted_list:
        if(tupla[0] == document):
          score_doc = score_doc + tupla[1]
    if(score_doc > 0):
      document_scores.append((score_doc, document))
  document_scores.sort(key = operator.itemgetter(0), reverse=True)
  docs_to_retrieve = document_scores[:k]
  return docs_to_retrieve
  

  
  
def termAtTimeRetrieval(query, index, k):
  query = query.lower() #Como na indexacao (no processo de tokenizacao) foi feito downcase, na consulta deve ser feito tbm
  hash_table = {}
  inverted_lists = []
  query_terms = query.split(" ")
  document_scores = [] #Uma lista de tuplas no formato (score_do_documento, documento)
  
  for term in query_terms:
    if term in index:
      inverted_lists.append(index[term])
  
  for inverted_list in inverted_lists:
    for post in inverted_list:
      if(post[0] in hash_table.keys()):
        hash_table[post[0]] = hash_table[post[0]] + post[1]
      else:
        hash_table[post[0]] = post[1]
  
  #Neste ultimo bloco de codigo:
  #Retornando valores na forma desejada (apenas por 'formatacao' do resultado - Lista de tuplas: [(doc, score)])
  for value in hash_table:
    document_scores.append((hash_table[value], value))
    
  document_scores.sort(key = operator.itemgetter(0), reverse=True)
  docs_to_retrieve = document_scores[:k]  
    
  return docs_to_retrieve



#Cria uma especie de novo indice atualizado, considerando apenas os top 'k' documentos da consulta 'query'
def update_index(query, k):
  
  top_docs = [doc for doc,score in termAtTimeRetrieval(query, indice, k)]
  top_docs_ranking = {} 
  
  for word in indice:
    top_docs_ranking[word] = []
    for doc_score in indice[word]:
      if doc_score[0] in top_docs:
        top_docs_ranking[word].append(doc_score)
        
        
  return top_docs_ranking


######################################
## Definindo consultas de um termo: ##
######################################
one_term_query = ["bolsonaro", "presidente", "lula", "golpe", "economia"]
    
    
  


**Considerando os top-3 documentos, aqui estão as lista das top-10 palavras associadas a cada consulta:**

**(Usando o algoritmo EMIM)**

In [138]:
#Agora o indice sera "atualizado" para aplicar os algoritmos de metricas de associacao
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('bolsonaro', 3)
top_words_using_emim_first_term = sorted(build_list_using_emim(consultas[0]), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Bolsonaro"': [x1 for (x1,x2) in top_words_using_emim_first_term]})


,"Consulta: ""Bolsonaro"""
0,hoje
1,presidente
2,existe
3,b
4,política
5,trabalho
6,democracia
7,criar
8,reforma
9,ainda


In [139]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('presidente', 3)
top_words_using_emim_second_term = sorted(build_list_using_emim('presidente'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Presidente" ': [x1 for (x1,x2) in top_words_using_emim_second_term]})

,"Consulta: ""Presidente"""
0,hoje
1,ainda
2,grande
3,existe
4,b
5,política
6,trabalho
7,democracia
8,criar
9,reforma


In [140]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('lula', 3)
top_words_using_emim_first_term = sorted(build_list_using_emim('lula'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Lula" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Lula"""
0,prefeito
1,rio
2,bispo
3,marcelo
4,crivella
5,achou
6,explicação
7,obsessão
8,evangélicos
9,israel


In [141]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('golpe', 3)
top_words_using_emim_first_term = sorted(build_list_using_emim('golpe'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Golpe" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Golpe"""
0,prefeito
1,rio
2,bispo
3,marcelo
4,crivella
5,achou
6,explicação
7,obsessão
8,evangélicos
9,israel


In [142]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('economia', 3)
top_words_using_emim_first_term = sorted(build_list_using_emim('economia'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Economia" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Economia"""
0,prefeito
1,rio
2,bispo
3,marcelo
4,crivella
5,achou
6,explicação
7,obsessão
8,evangélicos
9,israel


**Considerando os top-5 documentos, aqui estão as lista das top-10 palavras associadas a cada consulta:**

In [143]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('bolsonaro', 5)
top_words_using_emim_first_term = sorted(build_list_using_emim('bolsonaro'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Bolsonaro" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Bolsonaro"""
0,hoje
1,presidente
2,existe
3,b
4,política
5,democracia
6,criar
7,reforma
8,ainda
9,grande


In [144]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('presidente', 5)
top_words_using_emim_first_term = sorted(build_list_using_emim('presidente'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Presidente" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Presidente"""
0,hoje
1,ainda
2,grande
3,existe
4,b
5,política
6,democracia
7,criar
8,reforma
9,trabalho


In [145]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('lula', 5)
top_words_using_emim_first_term = sorted(build_list_using_emim('lula'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Lula" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Lula"""
0,prefeito
1,rio
2,bispo
3,marcelo
4,crivella
5,achou
6,explicação
7,obsessão
8,evangélicos
9,israel


In [146]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('golpe', 5)
top_words_using_emim_first_term = sorted(build_list_using_emim('golpe'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Golpe" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Golpe"""
0,procuradora
1,mortos
2,desaparecidos
3,políticos
4,goulart
5,forças
6,caminhada
7,ato
8,estudantes
9,regional


In [147]:
indice = build_index(lista_de_noticias) #Indice volta ao 'normal'
indice = update_index('economia', 5)
top_words_using_emim_first_term = sorted(build_list_using_emim('economia'), key = itemgetter(1), reverse = True)[0:10]
pd.DataFrame({'Consulta: "Economia" ': [x1 for (x1,x2) in top_words_using_emim_first_term]})

,"Consulta: ""Economia"""
0,prefeito
1,rio
2,bispo
3,marcelo
4,crivella
5,achou
6,explicação
7,obsessão
8,evangélicos
9,israel
